# Some Experiments about Tensor Spaces

In [ ]:
# FEniCSx libraries
from dolfinx.fem import FunctionSpace,assemble_scalar,form, Function
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import CellType,create_rectangle
from ufl import SpatialCoordinate,TestFunction,TrialFunction,inner,grad,div,sym,dx,as_vector
import ufl

# Other libraries
from mpi4py import MPI
import numpy as np
import pyvista
from dolfinx.plot import vtk_mesh
pyvista.set_jupyter_backend('client')

# Mesh ------------------------------------------------------------------------

msh = create_rectangle(comm=MPI.COMM_WORLD,
                       points=((0.0,0.0), (1.0,1.0)), n=(8,8),
                       cell_type=CellType.triangle)

x = SpatialCoordinate(msh)

# Variational problem ----------------------------------------------------------

# Some vector field
def ue1(x):
    return ufl.sin(ufl.pi*x[0])*ufl.sin(ufl.pi*x[1])

def ue2(x):
    return ufl.sin(ufl.pi*x[0])*ufl.sin(ufl.pi*x[1])

ue =  as_vector([ue1(x), ue2(x)])

# Tensor space
T_Ele = ufl.TensorElement("CG", msh.ufl_cell(),1,symmetry=True)
V = FunctionSpace(msh,T_Ele)

# Bilinear and linear forms
sig = TrialFunction(V)
tau = TestFunction(V)
a = inner(sig,tau)*dx
L = inner(sym(grad(ue)),tau)*dx

# Linear system
problem = LinearProblem(a,L,bcs=[],petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
wh = problem.solve()


In [ ]:
V_sub , map_ing = V.sub(0).collapse()
UVal = Function(V_sub)
UVal.x.array[:]=wh.x.array[map_ing]

In [ ]:
plotter = pyvista.Plotter()
vtkdata = vtk_mesh(msh, msh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
grid["mag"] =UVal.x.array
actor = plotter.add_mesh(grid, show_edges=True)

plotter.view_xy()
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:62906/index.html?ui=P_0x1075d2910_0&reconnect=auto' style='width: …